In [4]:
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import scipy as sc
# import cv2 as cv
import matplotlib.pyplot as plt
import rasterio as riot
from osgeo import gdal
import glacierml as gl
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
Image.MAX_IMAGE_PIXELS = None



2022-09-02 16:12:50.133377: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [10]:
ref = pd.read_csv('Farinotti_mean_thickness_RGI_ID_2.csv')
ref = df[[
    'RGIId',
    'Farinotti Mean Thickness'
]]
ref['region'] = ref['RGIId'].str[6:8]
ref = ref.sort_values('RGIId')
ref = ref.dropna()
ref

RGI = gl.RGI_loader()

RGI = RGI[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
#     'Zmed',
    'Zmax',
    'Area',
#     'Aspect',
    'Lmax'
]]
drops = RGI[

#     (RGI['Aspect'] < 0) | 
    (RGI['Zmin'] < 0) |
    (RGI['Zmax'] < 0) |
    (RGI['Lmax'] < 0) 

].index
RGI = RGI.drop(drops)
RGI['Zdelta'] = RGI['Zmax'] - RGI['Zmin']
RGI

ref = pd.merge(ref, RGI, left_index = True, right_index = True)
ref

,RGIId,Farinotti Mean Thickness,region
167121,RGI60-01.00001,21.182035,01
167122,RGI60-01.00002,30.519478,01
167123,RGI60-01.00003,35.518610,01
167124,RGI60-01.00004,53.077263,01
167125,RGI60-01.00005,47.519207,01
...,...,...,...
167116,RGI60-19.02748,12.948014,19
167117,RGI60-19.02749,22.894110,19
167118,RGI60-19.02750,60.874610,19
167119,RGI60-19.02751,10.053704,19


In [ ]:

dfy = pd.DataFrame()
for region_number in range(1,20,1):
    if len(str(region_number)) == 1:
        N = 1
        (region_number) = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
    dft = df[df['region'] == str(region_number)]
    
    extracted_count = len(dft)
    mean_thickness = dft['Farinotti Mean Thickness'].mean()
    median_thickness = dft['Farinotti Mean Thickness'].median()
    
    dfy = pd.concat([dfy, pd.Series(region_number, name = 'mean thickness')])
    dfy = dfy.reset_index()
    dfy = dfy.drop('index', axis = 1)
    # dfy = dfy.drop('0', axis = 1)
    dfy.loc[dfy.index[-1], 'region'] = region_number
    dfy.loc[dfy.index[-1], 'extracted mean'] = mean_thickness
    dfy.loc[dfy.index[-1], 'extracted median'] = median_thickness
    dfy.loc[dfy.index[-1], 'extracted count'] = extracted_count

dfy.loc[dfy.index[0], 'FR mean'] = 218
dfy.loc[dfy.index[1], 'FR mean'] = 72
dfy.loc[dfy.index[2], 'FR mean'] = 270
dfy.loc[dfy.index[3], 'FR mean'] = 210
dfy.loc[dfy.index[4], 'FR mean'] = 175
dfy.loc[dfy.index[5], 'FR mean'] = 341
dfy.loc[dfy.index[6], 'FR mean'] = 220
dfy.loc[dfy.index[7], 'FR mean'] = 101
dfy.loc[dfy.index[8], 'FR mean'] = 283
dfy.loc[dfy.index[9], 'FR mean'] = 56
dfy.loc[dfy.index[10], 'FR mean'] = 61
dfy.loc[dfy.index[11], 'FR mean'] = 48
dfy.loc[dfy.index[12], 'FR mean'] = 66
dfy.loc[dfy.index[13], 'FR mean'] = 85
dfy.loc[dfy.index[14], 'FR mean'] = 59
dfy.loc[dfy.index[15], 'FR mean'] = 42
dfy.loc[dfy.index[16], 'FR mean'] = 181
dfy.loc[dfy.index[17], 'FR mean'] = 63
dfy.loc[dfy.index[18], 'FR mean'] = 349


dfy.loc[dfy.index[0], 'FR area'] = 86677
dfy.loc[dfy.index[1], 'FR area'] = 14629
dfy.loc[dfy.index[2], 'FR area'] = 104920
dfy.loc[dfy.index[3], 'FR area'] = 40860
dfy.loc[dfy.index[4], 'FR area'] = 89651
dfy.loc[dfy.index[5], 'FR area'] = 11052
dfy.loc[dfy.index[6], 'FR area'] = 33932
dfy.loc[dfy.index[7], 'FR area'] = 2947
dfy.loc[dfy.index[8], 'FR area'] = 51551
dfy.loc[dfy.index[9], 'FR area'] = 2399
dfy.loc[dfy.index[10], 'FR area'] = 2091
dfy.loc[dfy.index[11], 'FR area'] = 1305
dfy.loc[dfy.index[12], 'FR area'] = 49295
dfy.loc[dfy.index[13], 'FR area'] = 33561
dfy.loc[dfy.index[14], 'FR area'] = 14734
dfy.loc[dfy.index[15], 'FR area'] = 2341
dfy.loc[dfy.index[16], 'FR area'] = 29368
dfy.loc[dfy.index[17], 'FR area'] = 1161
dfy.loc[dfy.index[18], 'FR area'] = 132771

dfy.loc[dfy.index[0], 'FR count'] = 27108
dfy.loc[dfy.index[1], 'FR count'] = 18862
dfy.loc[dfy.index[2], 'FR count'] = 4549
dfy.loc[dfy.index[3], 'FR count'] = 7422
dfy.loc[dfy.index[4], 'FR count'] = 19306
dfy.loc[dfy.index[5], 'FR count'] = 567
dfy.loc[dfy.index[6], 'FR count'] = 1615
dfy.loc[dfy.index[7], 'FR count'] = 3417
dfy.loc[dfy.index[8], 'FR count'] = 1069
dfy.loc[dfy.index[9], 'FR count'] = 5144
dfy.loc[dfy.index[10], 'FR count'] = 3927
dfy.loc[dfy.index[11], 'FR count'] = 1887
dfy.loc[dfy.index[12], 'FR count'] = 54429
dfy.loc[dfy.index[13], 'FR count'] = 27986
dfy.loc[dfy.index[14], 'FR count'] = 13119
dfy.loc[dfy.index[15], 'FR count'] = 2940
dfy.loc[dfy.index[16], 'FR count'] = 15908
dfy.loc[dfy.index[17], 'FR count'] = 3537
dfy.loc[dfy.index[18], 'FR count'] = 2751
dfy['FR mean'] = dfy['FR mean'].astype(int)

dfy['FR area'] = dfy['FR area'].astype(int)

dfy['FR count'] = dfy['FR count'].astype(int)
dfy['extracted count'] = dfy['extracted count'].astype(int)

dfy = dfy[[
    'region',
    'extracted mean',
    # 'extracted median',
    'FR mean',
    'FR area',
    'FR count',
    'extracted count'
]]
df = dfy
df

In [ ]:
df['"avg" area'] = df['FR area'] / df['FR count']
df['residual'] = df['extracted mean'] - df['FR mean']
df['ratio'] = df['extracted mean'] / df['FR mean']
df.sort_values('ratio', ascending = False)

In [ ]:
pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/'
# pth_2 = '/data/fast1/glacierml/data/RGI/rgi60-attribs/'
RGI_extra = pd.DataFrame(columns = ['Farinotti Mean Thickness'])
for file in tqdm(os.listdir(pth_2)):
    file_reader = pd.read_csv(pth_2 + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = pd.concat([RGI_extra, file_reader], ignore_index = True)

    # select only RGI data that was used to train the model   
RGI = RGI_extra[[
'RGIId',
    'Farinotti Mean Thickness'
]]
print(RGI)

In [ ]:


pth_1 = '/home/simonhans/data/prethicktor/RGI/outlines/Farinotti/'
# rootdir = '~'

df1 = pd.DataFrame()
df2 = pd.DataFrame()
for region_number in range(1,20,1)[:1]:
            
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
        
    region_folder = pth_1 + 'RGI60-' + str(region_number) + '/'
    for file in tqdm(os.listdir(region_folder)):
        
        RGIId = file[:14]
        
        raster = gdal.Open(region_folder + file)
        got = raster.GetGeoTransform()
        # print(got)
        pixelSizeX = got[1]
        pixelSizeY = -got[5]
        pixel_area = (pixelSizeX * pixelSizeY)

        im = Image.open(region_folder + file)
        imarray = np.array(im)
        df = pd.DataFrame(imarray)
        df = df.replace(0.0, np.nan) 
        
        volume = df * pixel_area
        
        # print(volume_1)
        # print(volume_2)
        total_volume = np.nansum(volume)
        
        area = np.count_nonzero(~np.isnan(df)) * pixel_area

        # thickness_1 = total_volume_1 / area_1
        # thickness_2 = total_volume_2 / area_2
        
        
        # mean_volume = sum(volume) / area

        total_volume = pd.Series(total_volume)
        df1 = df1.append(total_volume, ignore_index = True)
        df1.loc[df1.index[-1], 'Area'] = area / 1e6
        df1.loc[df1.index[-1], 'RGIId'] = RGIId
    
df1.to_csv('Farinotti_vol.csv'

        
        
        
    #     break
    # break
        

#         break
#     break

In [ ]:
df1

In [ ]:
f = (np.nansum(df1[0])) / 86677 / 1e6
# s = (np.nansum(df1['Vol 2'])) / 86677 / 1e6
sa = (np.nansum(df1['Area']))
print(f)

print('summed area = ' + str(sa / 1e6))
# print(s)
# r = np.nansum(df1['Area']) 
# q = np.nansum(df1['Area 2']) / 25 

# print(r)
# print(q)
# df7.mean()/86677
# print(df1)
# print('')
# print(df2)
# print('')

In [ ]:
df1['Area'] = df1['Area'] / 1e6
df1

In [ ]:
df1['Thickness'] = df1[0] / df1['Area']

In [ ]:
pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/'
RGI_extra = pd.DataFrame(columns = ['Farinotti Mean Thickness'])
for file in tqdm(os.listdir(pth_2)):
    file_reader = pd.read_csv(pth_2 + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = pd.concat([RGI_extra, file_reader], ignore_index = True)

In [ ]:
RGI_extra

In [ ]:
sum(df1[0])

In [ ]:
print('method 1 = ' + str(sum(df1[0]) / len[df1][0]))
print('method 2 = ' + str(sum(df2)))
print('max mean = ' + str(sum(df3)))
print('method 3 = ' + str(sum(df4)))
print('method 4 = ' + str(sum(df5)))

In [ ]:
df1

In [ ]:
pth_1 = '/home/simonhans/data/prethicktor/RGI/outlines/Millan/'
# rootdir = '~'
dft = pd.DataFrame()
for region_number in range(1,3,1)[:1]:
    print(region_number)
    # if len(str(region_number)) == 1:
    #     N = 1
    #     region_number = str(region_number).zfill(N + len(str(region_number)))
    # else:
    #     str(region_number) == str(region_number)
        
    region_folder = pth_1 + 'RGI-' + str(region_number) + '/'
    for file in (os.listdir(region_folder)[1:2]):

        print(file)
        im = Image.open(region_folder + file)
        im.show()
        imarray = np.array(im)
#         print(imarray.shape)
        df = pd.DataFrame(imarray)
        # print(df.head())
        df = df.replace(0.0, np.nan)
#         # print(df)

#         # mean_glacier_thickness_1 = df.mean().mean()

        mean_glacier_thickness_2 = pd.Series(np.nanmean(df.to_numpy()), name = 'thickness')
        dft = pd.concat(
            [dft, mean_glacier_thickness_2]
        )
#         # mean_by_hand = np.nansum(df) / np.count_nonzero(
#         #             ~np.isnan(df))

#         # print('mean by hand = ' + str(mean_by_hand))
#         # print('df.mean().mean() = ' + str(mean_glacier_thickness_1))
        # print('np.nanmean(df.to_numpy()) = ' + str(mean_glacier_thickness_2))
#         print('')
        # break
    #     break
    # break


# mean_glacier_thickness_1 = df.mean().mean()

# mean_glacier_thickness_2 = np.nanmean(df.to_numpy())

# mean by hand = np.nansum(
#     mean_glacier_thickness) / np.count_nonzero(~np.isnan(mean_glacier_thickness))

        
#         RGI['Farinotti Mean Thickness'].loc[RGI['RGIId'] == file[:14]] = mean_glacier_thickness
    #     break
    # break

In [ ]:
mean_glacier_thickness_2

In [ ]:
dft

In [ ]:
dft.mean() * 101